In [ ]:
import os
import functools

import cV3
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise


In [ ]:
def trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner'):
    action_space_ptp = env.action_space.high - env.action_space.low

    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        yield r, done, info, env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]

def writer_for(tag, fps, res):
    if not os.path.exists('movies'):
        os.mkdir('movies')
    return cV3.VideoWriter(
        f'movies/{tag}.avi',
        cV3.VideoWriter_fourcc('M','J','P','G'),
        fps,
        res
    )

In [ ]:
resolution = (1920, 1080)
camera = 'behindGripper' # one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('assembly-V3', np.zeros(4), 3, True),
    ('basketball-V3', np.zeros(4), 3, True),
    ('bin-picking-V3', np.zeros(4), 3, True),
    ('box-close-V3', np.zeros(4), 3, True),
    ('button-press-topdown-V3', np.zeros(4), 3, True),
    ('button-press-topdown-wall-V3', np.zeros(4), 3, True),
    ('button-press-V3', np.zeros(4), 3, True),
    ('button-press-wall-V3', np.zeros(4), 3, True),
    ('coffee-button-V3', np.zeros(4), 3, True),
    ('coffee-pull-V3', np.zeros(4), 3, True),
    ('coffee-push-V3', np.zeros(4), 3, True),
    ('dial-turn-V3', np.zeros(4), 3, True),
    ('disassemble-V3', np.zeros(4), 3, True),
    ('door-close-V3', np.zeros(4), 3, True),
    ('door-lock-V3', np.zeros(4), 3, True),
    ('door-open-V3', np.zeros(4), 3, True),
    ('door-unlock-V3', np.zeros(4), 3, True),
    ('hand-insert-V3', np.zeros(4), 3, True),
    ('drawer-close-V3', np.zeros(4), 3, True),
    ('drawer-open-V3', np.zeros(4), 3, True),
    ('faucet-open-V3', np.zeros(4), 3, True),
    ('faucet-close-V3', np.zeros(4), 3, True),
    ('hammer-V3', np.zeros(4), 3, True),
    ('handle-press-side-V3', np.zeros(4), 3, True),
    ('handle-press-V3', np.zeros(4), 3, True),
    ('handle-pull-side-V3', np.zeros(4), 3, True),
    ('handle-pull-V3', np.zeros(4), 3, True),
    ('lever-pull-V3', np.zeros(4), 3, True),
    ('peg-insert-side-V3', np.zeros(4), 3, True),
    ('pick-place-wall-V3', np.zeros(4), 3, True),
    ('pick-out-of-hole-V3', np.zeros(4), 3, True),
    ('reach-V3', np.zeros(4), 3, True),
    ('push-back-V3', np.zeros(4), 3, True),
    ('push-V3', np.zeros(4), 3, True),
    ('pick-place-V3', np.zeros(4), 3, True),
    ('plate-slide-V3', np.zeros(4), 3, True),
    ('plate-slide-side-V3', np.zeros(4), 3, True),
    ('plate-slide-back-V3', np.zeros(4), 3, True),
    ('plate-slide-back-side-V3', np.zeros(4), 3, True),
    ('peg-insert-side-V3', np.zeros(4), 3, True),
    ('peg-unplug-side-V3', np.zeros(4), 3, True),
    ('soccer-V3', np.zeros(4), 3, True),
    ('stick-push-V3', np.zeros(4), 3, True),
    ('stick-pull-V3', np.zeros(4), 3, True),
    ('push-wall-V3', np.zeros(4), 3, True),
    ('push-V3', np.zeros(4), 3, True),
    ('reach-wall-V3', np.zeros(4), 3, True),
    ('reach-V3', np.zeros(4), 3, True),
    ('shelf-place-V3', np.zeros(4), 3, True),
    ('sweep-into-V3', np.zeros(4), 3, True),
    ('sweep-V3', np.zeros(4), 3, True),
    ('window-open-V3', np.zeros(4), 3, True),
    ('window-close-V3', np.zeros(4), 3, True),
]

for env, noise, cycles, quit_on_success in config:
    tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)

    policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
    env = ALL_ENVS[env]()
    env._partially_observable = False
    env._freeze_rand_vec = False
    env._set_task_called = True

    writer = writer_for(tag, env.metadata['video.frames_per_second'], resolution)
    for _ in range(cycles):
        for r, done, info, img in trajectory_generator(env, policy, noise, resolution, camera):
            if flip: img = cV3.rotate(img, cV3.ROTATE_180)
            writer.write(img)
            if quit_on_success and info['success']:
                break

    writer.release()